In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==2.14

## Imports

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy
import tensorflow_datasets as tfds
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from scipy.linalg import sqrtm
from tqdm import tqdm

## Utils function

In [ ]:
import matplotlib.pyplot as plt

def display_images(images, captions=None):
  num_ver = len(images)//5 + 1
  num_hor = len(images) // num_ver +1
  plt.figure(figsize=(10,10))
  for i in range(len(images)):
    plt.subplot(num_ver,num_hor, i + 1)
    plt.imshow(images[i])
    plt.axis('off')

  plt.tight_layout()

## Global variables

In [ ]:
batch_size=10
shape = [299,299]

## Dataset

We are using Celeb_A dataset to measure a quality of images generated with pre-trained ProgressiveGAN model


In [ ]:
!wget -q -O celeba.zip https://seafile.unistra.fr/f/15dc03434ab245d2a960/?dl=1
!unzip -q celeba.zip

In [ ]:
import pandas as pd
import os

cx=89
cy=121
center = [128,128]

# Function to load and preprocess images
def load_and_preprocess_image(file_path):
    # Read and decode the image file
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    image = img[cy - 64 : cy + 64, cx - 64 : cx + 64]
    image = tf.image.resize(image,(shape[0],shape[1]))
    image = int(image)
    return image

# Load the list_eval_partition.csv file
csv_path = "list_eval_partition.csv"
partition_df = pd.read_csv(csv_path)
dataset_root = "img_align_celeba/img_align_celeba"
train_files = partition_df[partition_df['partition'] == 0]['image_id'].tolist()
train_paths = [os.path.join(dataset_root, file) for file in train_files]
# Create TensorFlow dataset for the training set
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.shuffle(1000)
train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=4)
train_dataset = train_dataset.batch(batch_size)

## Load pretrained Inception model

InceptionV3 model is used for feature extraction

In [ ]:
inception = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet',input_shape=(shape[0],shape[1],3),pooling='avg')

## Load pretrained Progressive GAN model

Instead of using your own GAN, here we will use already trained Progressive GAN in order to get intuition what FID score represents.

Later, you can load your own model instead.

In [ ]:
progan = hub.load("https://tfhub.dev/google/progan-128/1").signatures['default']

## Visualise datasets

### Visualise real dataset

In [ ]:
iterator= iter(train_dataset)
real_img = iterator.get_next()
display_images(real_img.numpy()/255)

###Visualise generated dataset

In [ ]:
#TODO:
noise = tf.random.normal([batch_size, 512])
fake_img = progan(noise)['default']*255
fake_img = tf.image.resize(fake_img,shape)

In [ ]:
display_images(fake_img.numpy()/255)

## Fréchet Distance


**Formula**

Based on the paper, "[The Fréchet distance between multivariate normal distributions](https://core.ac.uk/reader/82269844)" by Dowson and Landau (1982), the Fréchet distance between two multivariate normal distributions $X$ and $Y$ is:

$d(X, Y) = \Vert\mu_X-\mu_Y\Vert^2 + \mathrm{Tr}\left(\Sigma_X+\Sigma_Y - 2 \sqrt{\Sigma_X \Sigma_Y}\right)$

where $\mu_x$ and $\mu_y$ are means of corresponding distributions, and $\Sigma_X$ and $\Sigma_Y$ are c
ovariance  matrix

For calculating means, covariance  matrixes and traces, you can use **numpy library.**

*Read documentation carefully, some default paramters might not be what you want*


In [ ]:
# Function receives numpy arrays of pre-trained network's activations (e.g. real images and fake images activations)
# shape of activations is  (n_samples,2048) since we are taking last layer in Inception network
# mu1,mu2: the mean of the first/second  activations, shape = (n_features)
# sigma1,sigma2: the covariance matrix of the first/second activations, shape= (n_features, n_features)

def calculate_fid(act1,act2):
	# calculate mean and covariance statistics over
	#TODO:
	mu1 = act1.mean(axis=0)
	sigma1 = cov(act1, rowvar=False)
	mu2 = act2.mean(axis=0)
	sigma2 =  cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

### FID score between two sets of real images

In [ ]:
iterator= iter(train_dataset)
n_batches = 500
real_feature_list_1 = numpy.zeros((n_batches*batch_size,2048))
real_feature_list_2 = numpy.zeros((n_batches*batch_size,2048))
for i in tqdm(range(n_batches)):
  real_img = iterator.get_next()
  real_feat = inception(tf.keras.applications.inception_v3.preprocess_input(tf.cast(real_img,tf.float32)))
  real_feature_list_1[i*batch_size:(i+1)*batch_size] = real_feat.numpy()

  real_img = iterator.get_next()
  real_feat = inception(tf.keras.applications.inception_v3.preprocess_input(tf.cast(real_img,tf.float32)))
  real_feature_list_2[i*batch_size:(i+1)*batch_size] = real_feat.numpy()

In [ ]:
fid = calculate_fid(real_feature_list_1, real_feature_list_2)
print('FID: %.3f' % fid)

In [ ]:
fid = calculate_fid(real_feature_list_1, real_feature_list_1)
print('FID: %.3f' % fid)

### FID score between real and generated images

In [ ]:
n_batches = 500
real_feature_list = numpy.zeros((n_batches*batch_size,2048))
fake_feature_list = numpy.zeros((n_batches*batch_size,2048))
for i in tqdm(range(n_batches)):
  real_img = iterator.get_next()
  real_feat = inception(tf.keras.applications.inception_v3.preprocess_input(tf.cast(real_img,tf.float32)))
  real_feature_list[i*batch_size:(i+1)*batch_size] = real_feat.numpy()

  noise = tf.random.normal([batch_size, 512])
  #TODO..
  fake_img = progan(noise)['default']*255
  fake_img = tf.image.resize(fake_img,shape)
  fake_feat =inception(tf.keras.applications.inception_v3.preprocess_input(tf.cast(fake_img,tf.float32)))
  fake_feature_list[i*batch_size:(i+1)*batch_size] = fake_feat.numpy()




In [ ]:
fid = calculate_fid(real_feature_list, fake_feature_list)
print('FID: %.3f' % fid)

## Playground



1.   Experiment with different sample size (batch_size and n_batches)


